# Tekrarlayan Sinir Ağları

Önceki modülde, metnin zengin anlamsal temsillerini ve gömme katmanlarının üzerinde basit bir doğrusal sınıflandırıcı kullandık. Bu mimari, bir cümledeki kelimelerin toplu anlamını yakalar, ancak kelimelerin **sırasını** dikkate almaz çünkü gömme katmanlarının üzerindeki toplama işlemi, orijinal metinden bu bilgiyi kaldırır. Bu modeller kelime sıralamasını modelleyemediği için, metin oluşturma veya soru yanıtlama gibi daha karmaşık veya belirsiz görevleri çözemezler.

Metin dizisinin anlamını yakalamak için, **tekrarlayan sinir ağı** veya RNN olarak adlandırılan başka bir sinir ağı mimarisi kullanmamız gerekir. RNN'de, cümlemizi ağdan bir sembol bir sembol geçiririz ve ağ bir **durum** üretir, ardından bu durumu bir sonraki sembolle birlikte tekrar ağa geçiririz.

Verilen $X_0,\dots,X_n$ token dizisi girişine göre, RNN bir sinir ağı blokları dizisi oluşturur ve bu diziyi uçtan uca geri yayılım kullanarak eğitir. Her ağ bloğu bir çift $(X_i,S_i)$ alır ve sonuç olarak $S_{i+1}$ üretir. Son durum $S_n$ veya çıktı $X_n$, sonucu üretmek için doğrusal bir sınıflandırıcıya gider. Tüm ağ blokları aynı ağırlıkları paylaşır ve tek bir geri yayılım geçişiyle uçtan uca eğitilir.

Durum vektörleri $S_0,\dots,S_n$ ağdan geçtiği için, kelimeler arasındaki sıralı bağımlılıkları öğrenebilir. Örneğin, dizide bir yerde *not* kelimesi geçtiğinde, durum vektöründeki belirli öğeleri olumsuzlayarak olumsuzlama yapmayı öğrenebilir.

> Resimdeki tüm RNN bloklarının ağırlıkları paylaşıldığı için, aynı resim bir blok (sağda) olarak, ağın çıktı durumunu tekrar girişe geri döndüren bir tekrarlayan geri besleme döngüsü ile temsil edilebilir.

Haber veri setimizi sınıflandırmada tekrarlayan sinir ağlarının nasıl yardımcı olabileceğini görelim.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Basit RNN Sınıflandırıcı

Basit bir RNN durumunda, her tekrarlayan birim, birleştirilmiş giriş vektörünü ve durum vektörünü alarak yeni bir durum vektörü üreten basit bir doğrusal ağdır. PyTorch, bu birimi `RNNCell` sınıfıyla ve bu hücrelerin ağlarını ise `RNN` katmanı olarak temsil eder.

Bir RNN sınıflandırıcı tanımlamak için, önce giriş kelime dağarcığının boyutunu düşürmek amacıyla bir gömme katmanı uygulayacağız ve ardından bunun üzerine bir RNN katmanı ekleyeceğiz:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Not:** Burada basitlik açısından eğitilmemiş bir gömme katmanı kullanıyoruz, ancak daha iyi sonuçlar için önceki birimde açıklandığı gibi Word2Vec veya GloVe gömmeleriyle önceden eğitilmiş bir gömme katmanı kullanabiliriz. Daha iyi anlamak için bu kodu önceden eğitilmiş gömmelerle çalışacak şekilde uyarlamak isteyebilirsiniz.

Bizim durumumuzda, her bir grup aynı uzunlukta bir dizi doldurulmuş diziden oluşacak şekilde doldurulmuş bir veri yükleyici kullanacağız. RNN katmanı gömme tensörlerinin dizisini alacak ve iki çıktı üretecek:
* $x$, her adımda RNN hücre çıktılarının dizisidir
* $h$, dizinin son elemanı için nihai gizli durumdur

Sonrasında, sınıf sayısını elde etmek için tam bağlantılı bir doğrusal sınıflandırıcı uygularız.

> **Not:** RNN'leri eğitmek oldukça zordur, çünkü RNN hücreleri dizinin uzunluğu boyunca açıldığında, geri yayılımda yer alan katmanların sayısı oldukça fazla olur. Bu nedenle küçük bir öğrenme oranı seçmemiz ve iyi sonuçlar elde etmek için ağı daha büyük bir veri kümesinde eğitmemiz gerekir. Bu oldukça uzun sürebilir, bu yüzden GPU kullanımı tercih edilir.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Uzun Kısa Süreli Bellek (LSTM)

Klasik RNN'lerin en büyük problemlerinden biri, **kaybolan gradyanlar** problemidir. RNN'ler tek bir geri yayılım geçişinde uçtan uca eğitildiği için, hatayı ağın ilk katmanlarına iletmekte zorlanır ve bu nedenle ağ, uzak tokenler arasındaki ilişkileri öğrenemez. Bu sorunu önlemenin yollarından biri, **kapılar** olarak adlandırılan **açık durum yönetimi**ni tanıtmaktır. Bu türdeki en bilinen iki mimari şunlardır: **Uzun Kısa Süreli Bellek** (LSTM) ve **Gated Relay Unit** (GRU).

![Uzun kısa süreli bellek hücresine örnek gösteren bir görsel](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM Ağı, RNN'e benzer bir şekilde organize edilmiştir, ancak katmandan katmana geçen iki durum vardır: gerçek durum $c$ ve gizli vektör $h$. Her birimde, gizli vektör $h_i$ giriş $x_i$ ile birleştirilir ve **kapılar** aracılığıyla durum $c$ üzerinde ne olacağını kontrol eder. Her kapı, sigmoid aktivasyonuna sahip bir sinir ağıdır (çıktı aralığı $[0,1]$), ve durum vektörüyle çarpıldığında bit düzeyinde bir maske olarak düşünülebilir. Yukarıdaki resimde soldan sağa doğru şu kapılar bulunmaktadır:
* **unutma kapısı**, gizli vektörü alır ve vektör $c$'nin hangi bileşenlerini unutmamız gerektiğini ve hangilerini geçirmemiz gerektiğini belirler.
* **giriş kapısı**, girişten ve gizli vektörden bazı bilgileri alır ve duruma ekler.
* **çıkış kapısı**, durumu $\tanh$ aktivasyonu ile bir doğrusal katman aracılığıyla dönüştürür, ardından yeni durum $c_{i+1}$ üretmek için gizli vektör $h_i$ kullanarak bazı bileşenlerini seçer.

Durum $c$'nin bileşenleri, açılıp kapatılabilen bayraklar olarak düşünülebilir. Örneğin, bir dizide *Alice* adını gördüğümüzde, bunun bir kadın karaktere atıfta bulunduğunu varsayabilir ve cümlede kadın bir isim olduğunu belirten bir bayrağı duruma yükseltebiliriz. Daha sonra *ve Tom* ifadelerini gördüğümüzde, çoğul bir isim olduğunu belirten bir bayrağı yükseltebiliriz. Böylece, durumu manipüle ederek cümle parçalarının dilbilgisel özelliklerini takip edebiliriz.

> **Not**: LSTM'nin iç yapısını anlamak için harika bir kaynak, Christopher Olah'ın [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) adlı makalesidir.

LSTM hücresinin iç yapısı karmaşık görünse de, PyTorch bu implementasyonu `LSTMCell` sınıfı içinde gizler ve tüm LSTM katmanını temsil etmek için `LSTM` nesnesini sağlar. Bu nedenle, LSTM sınıflandırıcısının implementasyonu, yukarıda gördüğümüz basit RNN'e oldukça benzer olacaktır:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Paketlenmiş Diziler

Örneğimizde, minibatch içindeki tüm dizileri sıfır vektörleriyle doldurmak zorunda kaldık. Bu, bir miktar bellek israfına yol açsa da, RNN'ler için daha kritik olan, doldurulmuş giriş öğeleri için ek RNN hücrelerinin oluşturulmasıdır. Bu hücreler eğitim sürecine katılır, ancak önemli bir giriş bilgisi taşımaz. RNN'yi yalnızca gerçek dizi boyutuna göre eğitmek çok daha iyi olurdu.

Bunu yapmak için, PyTorch'ta doldurulmuş dizilerin saklanması için özel bir format tanıtılmıştır. Diyelim ki doldurulmuş bir minibatch girişimiz şu şekilde görünüyor:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Burada 0 doldurulmuş değerleri temsil eder ve giriş dizilerinin gerçek uzunluk vektörü `[5,3,1]` şeklindedir.

Doldurulmuş dizilerle RNN'yi etkili bir şekilde eğitmek için, RNN hücrelerinin ilk grubunu büyük bir minibatch ile (`[1,6,9]`) eğitmeye başlamak, ardından üçüncü dizinin işlemini sonlandırmak ve daha kısa minibatch'lerle (`[2,7]`, `[3,8]`) eğitime devam etmek istiyoruz. Bu şekilde, paketlenmiş dizi tek bir vektör olarak temsil edilir - bizim durumumuzda `[1,6,9,2,7,3,8,4,5]`, ve uzunluk vektörü (`[5,3,1]`), bu vektörden orijinal doldurulmuş minibatch'i kolayca yeniden oluşturabiliriz.

Paketlenmiş dizi oluşturmak için `torch.nn.utils.rnn.pack_padded_sequence` fonksiyonunu kullanabiliriz. RNN, LSTM ve GRU dahil olmak üzere tüm tekrarlayan katmanlar, giriş olarak paketlenmiş dizileri destekler ve paketlenmiş çıktı üretir. Bu çıktı, `torch.nn.utils.rnn.pad_packed_sequence` kullanılarak çözülebilir.

Paketlenmiş dizi üretebilmek için, uzunluk vektörünü ağa iletmemiz gerekir ve bu nedenle minibatch'leri hazırlamak için farklı bir fonksiyona ihtiyacımız vardır:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Gerçek ağ, yukarıdaki `LSTMClassifier` ile çok benzer olacaktır, ancak `forward` geçişi hem doldurulmuş minibatch'i hem de dizi uzunluklarının vektörünü alacaktır. Gömülü temsili hesapladıktan sonra, paketlenmiş diziyi hesaplar, bunu LSTM katmanına geçirir ve ardından sonucu tekrar açarız.

> **Not**: Aslında açılmış sonuç `x`'i kullanmıyoruz, çünkü sonraki hesaplamalarda gizli katmanlardan gelen çıktıyı kullanıyoruz. Bu nedenle, bu koddan açma işlemini tamamen kaldırabiliriz. Burada yerleştirme sebebimiz, ağ çıktısını daha sonraki hesaplamalarda kullanmanız gerektiğinde bu kodu kolayca değiştirebilmenizdir.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Not:** Eğitim fonksiyonuna geçirdiğimiz `use_pack_sequence` parametresini fark etmiş olabilirsiniz. Şu anda, `pack_padded_sequence` fonksiyonu, uzunluk dizisi tensörünün CPU cihazında olmasını gerektiriyor ve bu nedenle eğitim fonksiyonu, eğitim sırasında uzunluk dizisi verilerini GPU'ya taşımaktan kaçınmalıdır. [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) dosyasındaki `train_emb` fonksiyonunun implementasyonuna göz atabilirsiniz.


## Çift Yönlü ve Çok Katmanlı RNN'ler

Örneklerimizde, tüm tekrarlayan ağlar bir yönde çalıştı; bir dizinin başından sonuna doğru. Bu doğal görünüyor, çünkü okuma ve konuşmayı dinleme şeklimizi andırıyor. Ancak, birçok pratik durumda giriş dizisine rastgele erişimimiz olduğu için, tekrarlayan hesaplamayı her iki yönde de çalıştırmak mantıklı olabilir. Bu tür ağlara **çift yönlü** RNN'ler denir ve RNN/LSTM/GRU yapıcısına `bidirectional=True` parametresini geçirerek oluşturulabilirler.

Çift yönlü bir ağ ile çalışırken, her yön için birer tane olmak üzere iki gizli durum vektörüne ihtiyacımız olur. PyTorch, bu vektörleri iki kat daha büyük bir boyutta tek bir vektör olarak kodlar, bu oldukça kullanışlıdır çünkü genellikle ortaya çıkan gizli durumu tam bağlantılı bir doğrusal katmana geçirirsiniz ve katmanı oluştururken bu boyut artışını dikkate almanız yeterlidir.

Tekrarlayan ağlar, tek yönlü veya çift yönlü olsun, bir dizideki belirli desenleri yakalar ve bunları durum vektörüne kaydedebilir veya çıktıya aktarabilir. Konvolüsyonel ağlarda olduğu gibi, birinci katman tarafından çıkarılan düşük seviyeli desenlerden daha yüksek seviyeli desenleri yakalamak için birinci katmanın üzerine başka bir tekrarlayan katman inşa edebiliriz. Bu bizi, bir önceki katmanın çıktısının bir sonraki katmana giriş olarak geçtiği, iki veya daha fazla tekrarlayan ağdan oluşan **çok katmanlı RNN** kavramına götürür.

![Çok Katmanlı Uzun-Kısa Süreli Bellek RNN'yi gösteren bir görsel](../../../../../translated_images/tr/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Fernando López'in [bu harika yazısından](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) alınmış bir görsel*

PyTorch, bu tür ağları oluşturmayı kolaylaştırır, çünkü RNN/LSTM/GRU yapıcısına `num_layers` parametresini geçirerek birkaç tekrarlama katmanını otomatik olarak oluşturabilirsiniz. Bu aynı zamanda gizli/durum vektörünün boyutunun orantılı olarak artacağı anlamına gelir ve tekrarlayan katmanların çıktısını işlerken bunu dikkate almanız gerekir.


## Diğer Görevler için RNN'ler

Bu bölümde, RNN'lerin dizi sınıflandırması için kullanılabileceğini gördük, ancak aslında metin oluşturma, makine çevirisi ve daha fazlası gibi birçok başka görevi de yerine getirebilirler. Bu görevleri bir sonraki bölümde ele alacağız.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul edilmez.
